<a href="https://colab.research.google.com/github/aiml-hrushikesh/Research-Paper-Analysis-and-Classification-Velsera/blob/main/Question_1_and_3_Research_Paper_Analysis_%26_Classification_Pipeline_Velsera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install and import the necessary libraries
!pip install torch
!pip install -q -U accelerate peft bitsandbytes transformers trl einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [19]:
pip install -U huggingface_hub


In [20]:
from huggingface_hub import login
login()


In [21]:
# Standard libraries for data processing
import os
import re
import numpy as np
import pandas as pd
import torch
from huggingface_hub import HfApi, create_repo, upload_folder

# For evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# Hugging Face dataset utility
from datasets import Dataset

# Transformers and PEFT modules
from transformers import (
    AutoModelForSequenceClassification,   # For classification task
    AutoTokenizer,
    BitsAndBytesConfig,                   # For quantization config
    TrainingArguments,
    Trainer                               # Huggingface trainer class
)

from peft import (
    LoraConfig,                           # LoRA fine-tuning config
    prepare_model_for_kbit_training,     # Prepares model for low-bit training
    get_peft_model,                      # Applies PEFT on base model
    TaskType                             # Specifies the training task type
)


In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"


# ---------------------- Step 1: Load & Preprocess Data ----------------------

In [9]:
# Load dataset from CSV
df = pd.read_csv("/content/fine_tuning_data.csv")

# Drop rows with missing text values
df.dropna(subset=["text"], inplace=True)


def clean_text(text):
    text = str(text)
    text = text.replace("Title:", "")         # Remove 'Title:' label
    text = text.replace("Abstract:", "")      # Remove 'Abstract:' label
    text = re.sub(r'\[[0-9]+\]', '', text)    # Remove citations like [1]
    text = re.sub(r'\(.*?\)', '', text)       # Remove (Doe et al., 2020)
    text = re.sub(r'[^A-Za-z0-9., ]+', ' ', text)  # Keep alphanumerics and common punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    return text

In [11]:
# Apply cleaning function
df["cleaned_text"] = df["text"].apply(clean_text)

# Convert labels to binary: Cancer = 1, Non-Cancer = 0
df["binary_label"] = df["label"].map({"Cancer": 1, "Non-Cancer": 0})

# Keep only necessary columns
df = df[["ID","cleaned_text", "binary_label"]]
df

,ID,cleaned_text,binary_label
0,30872385,Comparison of methodologies for the detection ...,1
1,30873683,Tumour biomarkers Tracing the molecular functi...,1
2,30874851,"Pomalidomide, cyclophosphamide, and dexamethas...",1
3,30875581,Aggressive variants of prostate cancer Are we ...,1
4,30875950,"Circulating Tumour Cells , Head and Neck Cance...",1
...,...,...,...
995,38623902,Not Available . Effective longitudinal biomark...,0
996,38640937,Mechanisms and management of loss of response ...,0
997,38642556,Modification of coronary artery disease clinic...,0
998,38650020,Meta analysis of the global distribution of cl...,0


# ---------------------- Step 2: Prepare HuggingFace Dataset ----------------------


In [12]:
# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Rename columns to match Hugging Face expectations
dataset = dataset.map(lambda x: {"text": str(x["cleaned_text"]), "label": int(x["binary_label"])})

# Remove redundant columns
dataset = dataset.remove_columns(["cleaned_text", "binary_label"])

# Train-test split (80-20)
dataset = dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 200
    })
})

# ---------------------- Step 3: Tokenization ----------------------

In [14]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Step 3: Load Model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# ---------------------- Step 4: LoRA Configuration ----------------------


In [15]:
# Load classification model with quantization (4-bit inference)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    task_type=TaskType.SEQ_CLS,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_lin", "v_lin"]  # These are relevant for DistilBERT
)

# Step 5: Apply LoRA
model = get_peft_model(base_model, lora_config)

# ---------------------- Step 6: Define Training Arguments ----------------------

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    report_to="none"  # disables wandb
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# ---------------------- Step 7: Train with HuggingFace Trainer ----------------------


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


# Start training
trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,0.598573
2,No log,0.284739
3,No log,0.209630


TrainOutput(global_step=300, training_loss=0.453898671468099, metrics={'train_runtime': 90.8461, 'train_samples_per_second': 26.418, 'train_steps_per_second': 3.302, 'total_flos': 323374576435200.0, 'train_loss': 0.453898671468099, 'epoch': 3.0})

Save the Fine-Tuned Model

In [18]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")


('finetuned_model/tokenizer_config.json',
 'finetuned_model/special_tokens_map.json',
 'finetuned_model/vocab.txt',
 'finetuned_model/added_tokens.json',
 'finetuned_model/tokenizer.json')

Pushing Model to Hugging face hub

In [23]:


repo_name = "cancer-classifier-distilbert"
model_path = "finetuned_model"
hf_username = "hkchavan"  # replace with your Hugging Face username

full_repo_id = f"{hf_username}/{repo_name}"

# Create repo (if not already created)
create_repo(repo_id=full_repo_id, exist_ok=True)

# Upload model folder to repo
upload_folder(
    folder_path=model_path,
    repo_id=full_repo_id,
    path_in_repo=".",  # root of repo
    repo_type="model"
)

print(f"✅ Model uploaded: https://huggingface.co/{full_repo_id}")


adapter_model.safetensors:   0%|          | 0.00/2.96M [00:00<?, ?B/s]

✅ Model uploaded: https://huggingface.co/hkchavan/cancer-classifier-distilbert


# **3. Model Performance Evaluation**

•	Baseline Model Performance:

In [24]:
base_trainer = Trainer(model=base_model)

#  Predict on tokenized test set
base_outputs = base_trainer.predict(tokenized_datasets["test"])

# Step 4: Predictions and True Labels
base_preds = np.argmax(base_outputs.predictions, axis=1)
true = base_outputs.label_ids

# Step 5: Report Metrics
print("📊 Baseline Classification Report:\n", classification_report(true, base_preds))
print("🧾 Baseline Confusion Matrix:\n", confusion_matrix(true, base_preds))

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


📊 Baseline Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.98      0.94        97
           1       0.98      0.89      0.93       103

    accuracy                           0.94       200
   macro avg       0.94      0.94      0.93       200
weighted avg       0.94      0.94      0.93       200

🧾 Baseline Confusion Matrix:
 [[95  2]
 [11 92]]


•	Fine-Tuned Model Performance:

In [25]:
# Use tokenized test set
outputs = trainer.predict(tokenized_datasets["test"])

# Predictions and true labels
preds = np.argmax(outputs.predictions, axis=1)
true = outputs.label_ids

# Report
print("Classification Report:\n", classification_report(true, preds))
print("Confusion Matrix:\n", confusion_matrix(true, preds))

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.98      0.94        97
           1       0.98      0.89      0.93       103

    accuracy                           0.94       200
   macro avg       0.94      0.94      0.93       200
weighted avg       0.94      0.94      0.93       200

Confusion Matrix:
 [[95  2]
 [11 92]]


# **1.	1. Multi-Label Classification Output**

In [30]:
# Load model and tokenizer from Hugging Face Hub
model = AutoModelForSequenceClassification.from_pretrained("hkchavan/cancer-classifier-distilbert")
tokenizer = AutoTokenizer.from_pretrained("hkchavan/cancer-classifier-distilbert")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Sample abstract
abstract_text = """Title: Pancreatic acinar cell carcinoma is associated with BRCA2 germline mutations: a case report and literature review. Abstract: Acinar cell carcinoma (ACC) is a rare pancreatic neoplasm with dismal prognosis. Insights into the molecular basis of ACC can pave the way for the application of more effective, personalized therapies and detection of patients with hereditary predisposition. Molecular analysis revealed a germline BRCA2 (and CHEK2) mutation in a patient with a rare pancreatic ACC with extensive intraductal growth. Somatic loss of the wild-type BRCA2 allele in the tumor indicated the causal relationship of ACC with the germline defect. A thorough literature review identified another nine ACCs associated with germline BRCA2 mutation and two ACCs associated with germline BRCA1 mutation, resulting in a prevalence of BRCA1/2 germline mutations in almost 7% of ACCs. Moreover, somatic BRCA1/2 alterations are reported in 16% of sporadic ACCs. Overall, about one fifth (22%) of all pancreatic ACCs exhibit BRCA1/2 deficiency. This study underscores the important role of BRCA1/2 mutations in pancreatic ACC. All ACC patients should undergo genetic testing for BRCA1/2 mutations to identify carriers of pathogenic variants. This will allow to select patients that can benefit from targeted therapies directed against BRCA1/2-deficient tumors and is also crucial as a referral to genetic screening for the relatives of affected individuals carrying germline BRCA1/2 alterations. Abbreviations: ACC: acinar cell carcinoma; HBOC: Hereditary Breast and Ovarian Cancer; LOH: loss of heterozygosity; PARP: poly (ADP-ribose) polymerase; PDAC: pancreatic ductal adenocarcinoma; PP: pancreatic panniculitis; SD: standard deviation; WES: whole-exome sequencing."""

# Tokenize and move inputs to the same device
inputs = tokenizer(abstract_text, return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = F.softmax(logits, dim=1).squeeze().tolist()

# Label mapping
id2label = {0: "Non-Cancer", 1: "Cancer"}

# Thresholding and formatting output
predicted_label_ids = [i for i, p in enumerate(probs) if p > 0.5]
predicted_labels = [id2label[i] for i in predicted_label_ids]
confidence_scores = {id2label[i]: round(p, 2) for i, p in enumerate(probs)}

# Final Output
output = {
    "predicted_labels": predicted_labels,
    "confidence_scores": confidence_scores
}

import json
print(json.dumps(output, indent=2))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{
  "predicted_labels": [
    "Cancer"
  ],
  "confidence_scores": {
    "Non-Cancer": 0.05,
    "Cancer": 0.95
  }
}
